In [30]:
!apt-get install libzbar0

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libzbar0 is already the newest version (0.23.92-4build2).
0 upgraded, 0 newly installed, 0 to remove and 122 not upgraded.


In [31]:
# Step 1: Installing required packages
!pip install opencv-python --quiet
!pip install python-barcode --quiet
!pip install pyzbar --quiet
!pip install matplotlib --quiet

In [32]:
# Step 2: Importing libraries
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from IPython.display import Image, display
import barcode
from barcode.writer import ImageWriter
import os
from pyzbar.pyzbar import decode
import matplotlib.pyplot as plt

In [33]:
# Step 3: Loading dataset
df = pd.read_csv("/kaggle/input/world-food-facts/en.openfoodfacts.org.products.tsv", 
                 sep='\t', low_memory=False)

In [34]:
# Step 4: Keeping only required columns and shortening dataset
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

columns_needed = [
    'code', 'product_name', 'brands', 'ingredients_text', 'carbohydrates_100g',
    'fat_100g', 'fiber_100g', 'proteins_100g', 'salt_100g', 'sugars_100g'
]

df = df[[col for col in columns_needed if col in df.columns]].dropna(subset=['code', 'product_name'])
df = df[df.isna().sum(axis=1) <= 2].head(10_000)
# Preview
df.head()

,code,product_name,brands,ingredients_text,carbohydrates_100g,fat_100g,fiber_100g,proteins_100g,salt_100g,sugars_100g
1,0000000004530,Banana Chips Sweetened (Whole),NaN,"Bananas, vegetable oil (coconut oil, corn oil ...",64.29,28.57,3.6,3.57,0.00000,14.29
2,0000000004559,Peanuts,Torn & Glasser,"Peanuts, wheat flour, sugar, rice flour, tapio...",60.71,17.86,7.1,17.86,0.63500,17.86
3,0000000016087,Organic Salted Nut Mix,Grizzlies,"Organic hazelnuts, organic cashews, organic wa...",17.86,57.14,7.1,17.86,1.22428,3.57
4,0000000016094,Organic Polenta,Bob's Red Mill,Organic polenta,77.14,1.43,5.7,8.57,NaN,NaN
5,0000000016100,Breadshop Honey Gone Nuts Granola,Unfi,"Rolled oats, grape concentrate, expeller press...",63.46,18.27,7.7,13.46,NaN,11.54


In [35]:
# Step 5: Defining a function to look up barcode and show product info
def lookup_barcode(barcode):
    product = df[df['code'] == str(barcode)]
    if not product.empty:
        data = product.iloc[0]
        print(f"📦 Product       : {data['product_name']}")
        print(f"🏢 Brand         : {data.get('brands', 'N/A')}")
        print(f"🧾 Ingredients   : {data.get('ingredients_text', 'Not listed')}")
        print(f"🍞 Carbs (100g)  : {data.get('carbohydrates_100g', 'N/A')}")
        print(f"💪 Fat (100g)    : {data.get('fat_100g', 'N/A')}")
        print(f"🌾 Fiber (100g)  : {data.get('fiber_100g', 'N/A')}")
        print(f"🥩 Protein (100g): {data.get('proteins_100g', 'N/A')}")
        print(f"🧂 Salt (100g)   : {data.get('salt_100g', 'N/A')}")
        print(f"🍬 Sugar (100g)  : {data.get('sugars_100g', 'N/A')}")  
        return data
    else:
        print("❌ Barcode not found.")
        return None

In [40]:
# Step 6: Trying with a sample barcode
image_path = '/kaggle/input/da623-barcodes/0000000030564.png'  # make sure the filename is correct
image = cv2.imread(image_path)

if image is None:
    print(f"❌ Could not read image from {image_path}")
else:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detecting barcodes in the grayscale image
    barcodes = decode(gray)

    if barcodes:
        barcode_data = barcodes[0].data.decode("utf-8")
        print("📷 Detected Barcode:", barcode_data)

        # Lookup product information
        lookup_barcode(barcode_data)
    else:
        print("🚫 No barcode detected in the image.")

📷 Detected Barcode: 0000000030564
📦 Product       : Organic Small Red Chili Beans
🏢 Brand         : Unfi
🧾 Ingredients   : High fiber, low fat, vegetable nourishment.
🍞 Carbs (100g)  : 63.89
💪 Fat (100g)    : nan
🌾 Fiber (100g)  : 44.4
🥩 Protein (100g): 22.22
🧂 Salt (100g)   : 0.17526
🍬 Sugar (100g)  : 2.78
